<a href="https://colab.research.google.com/github/marymlucas/obesity_lifestyle_diet/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project - Predict obesity of individuals based on diet and lifestyle habits

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [3]:
!wget https://dlcdn.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz

--2022-03-04 19:03:41--  https://dlcdn.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 220400553 (210M) [application/x-gzip]
Saving to: ‘spark-3.0.3-bin-hadoop2.7.tgz’

spark-3.0.3-bin-had 100%[===================>] 210.19M   227MB/s    in 0.9s    

2022-03-04 19:03:42 (227 MB/s) - ‘spark-3.0.3-bin-hadoop2.7.tgz’ saved [220400553/220400553]



In [4]:
!tar xf spark-3.0.3-bin-hadoop2.7.tgz

In [5]:
!pip install -q findspark

In [6]:
!pip install pyspark==3.0.3

     |████████████████████████████████| 209.1 MB 51 kB/s 
     |████████████████████████████████| 198 kB 50.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.3-py2.py3-none-any.whl size=209435971 sha256=a7a4b652879d43cfaab76eedade21b76891d81aee22369c8de9e12671251eec5
  Stored in directory: /root/.cache/pip/wheels/7e/6d/0a/6b0bf301bc056d9af03194b732b9f49ad2fceb205aab2984fd
Successfully built pyspark


In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.0.3-bin-hadoop2.7"

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
from pyspark.sql import SparkSession

In [10]:
spark = SparkSession.builder.appName('DSCI632 - Final Project').getOrCreate()

In [11]:
import pandas as pd
import numpy as np

# Data Import and Exploration

In [12]:
raw_data = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load('/content/drive/MyDrive/Colab Notebooks/DSCI-632/project/data/ObesityDataSet_raw_and_data_sinthetic.csv')

In [13]:
raw_data.show(5)

+------+----+------+------+------------------------------+----+----+---+---------+-----+----+---+---+---+----------+--------------------+-------------------+
|Gender| Age|Height|Weight|family_history_with_overweight|FAVC|FCVC|NCP|     CAEC|SMOKE|CH2O|SCC|FAF|TUE|      CALC|              MTRANS|         NObeyesdad|
+------+----+------+------+------------------------------+----+----+---+---------+-----+----+---+---+---+----------+--------------------+-------------------+
|Female|21.0|  1.62|  64.0|                           yes|  no| 2.0|3.0|Sometimes|   no| 2.0| no|0.0|1.0|        no|Public_Transporta...|      Normal_Weight|
|Female|21.0|  1.52|  56.0|                           yes|  no| 3.0|3.0|Sometimes|  yes| 3.0|yes|3.0|0.0| Sometimes|Public_Transporta...|      Normal_Weight|
|  Male|23.0|   1.8|  77.0|                           yes|  no| 2.0|3.0|Sometimes|   no| 2.0| no|2.0|1.0|Frequently|Public_Transporta...|      Normal_Weight|
|  Male|27.0|   1.8|  87.0|                         

In [14]:
# the data attributes and types are in the top part of the arff file
!pip install liac-arff

  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11732 sha256=0800f329968a96ace29dda54a76356a97401a67a54420a94026baa59292e74c7
  Stored in directory: /root/.cache/pip/wheels/1f/0f/15/332ca86cbebf25ddf98518caaf887945fbe1712b97a0f2493b
Successfully built liac-arff


In [15]:
import arff

with open('/content/drive/MyDrive/Colab Notebooks/DSCI-632/project/data/ObesityDataSet_raw_and_data_sinthetic.arff') as handle:
  data = arff.load(handle)

#print(data['attributes'])
for attribute in data['attributes']:
  print(attribute)

('Gender', ['Female', 'Male'])
('Age', 'NUMERIC')
('Height', 'NUMERIC')
('Weight', 'NUMERIC')
('family_history_with_overweight', ['yes', 'no'])
('FAVC', ['yes', 'no'])
('FCVC', 'NUMERIC')
('NCP', 'NUMERIC')
('CAEC', ['no', 'Sometimes', 'Frequently', 'Always'])
('SMOKE', ['yes', 'no'])
('CH2O', 'NUMERIC')
('SCC', ['yes', 'no'])
('FAF', 'NUMERIC')
('TUE', 'NUMERIC')
('CALC', ['no', 'Sometimes', 'Frequently', 'Always'])
('MTRANS', ['Automobile', 'Motorbike', 'Bike', 'Public_Transportation', 'Walking'])
('NObeyesdad', ['Insufficient_Weight', 'Normal_Weight', 'Overweight_Level_I', 'Overweight_Level_II', 'Obesity_Type_I', 'Obesity_Type_II', 'Obesity_Type_III'])


## EXPLORATORY DATA ANALYSIS

In [16]:
raw_data.printSchema()

root
 |-- Gender: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Height: double (nullable = true)
 |-- Weight: double (nullable = true)
 |-- family_history_with_overweight: string (nullable = true)
 |-- FAVC: string (nullable = true)
 |-- FCVC: double (nullable = true)
 |-- NCP: double (nullable = true)
 |-- CAEC: string (nullable = true)
 |-- SMOKE: string (nullable = true)
 |-- CH2O: double (nullable = true)
 |-- SCC: string (nullable = true)
 |-- FAF: double (nullable = true)
 |-- TUE: double (nullable = true)
 |-- CALC: string (nullable = true)
 |-- MTRANS: string (nullable = true)
 |-- NObeyesdad: string (nullable = true)

